In [ ]:
from app_figures import Data,Figures
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np



In [ ]:
data=Data()
data.data_stations_2021.head()

Create dict {station : deplacements_per_month(dataframe)}

In [ ]:
df=data.data_stations_2021[data.data_stations_2021["emplacement_pk_start"]==10].groupby("Month").count()["start_date"]
df.index
df.values

In [ ]:
df_pk_group=data.data_stations_2021.groupby("emplacement_pk_start")
my_dict={}
for pk,df in df_pk_group:
    df=df.groupby("Month").count().filter(items=["start_date"])
    df.rename({"start_date":"nb_trajets"},inplace=True)
    my_dict[pk]=df
my_dict

In [ ]:
import plotly.graph_objects as go
station_list=["Métro Angrignon (Lamont /  des Trinitaires)"]
df=data.bixi_stations
station_list_id=[df.loc[df["name"]==name]["pk"].values[0] for name in station_list]
station_list_id
my_dict_with_name={station_name:my_dict[station_list_id[idx]] for idx,station_name in enumerate(station_list)}
my_dict_with_name

fig=go.Figure()
for station_name,df in my_dict_with_name.items():
    fig.add_trace(go.Scatter(x=df.index,y=df["start_date"],name=station_name))
fig.show()



In [ ]:
data.bixi_stations.head()

Create deplacement for all stations

In [ ]:
df=pd.read_csv("data/deplacement.csv")
# Drop rows where same pickup and dropoff location
index_to_drop=df[(df["pickup_latitude"]==df["dropoff_latitude"]) | (df["pickup_longitude"]==df["dropoff_longitude"])].index
df.drop(index_to_drop,inplace=True)
df
# df[df["pickup_name"]=="Métro Angrignon (Lamont /  des Trinitaires)"]

In [ ]:
import matplotlib as mpl

def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1="#22c1c3"
c2="#fdbb2d"

fig = go.Figure()
# Impossible to trace all data
# Try per station
pickup_name = "Métro Angrignon (Lamont /  des Trinitaires)"
min_max_width = [1, 4]
min_max_nb = [
    df[df["pickup_name"] == pickup_name]["nb"].min(),
    df[df["pickup_name"] == pickup_name]["nb"].max(),
]
# print(np.interp(33,min_max_nb,min_max_width))
for idx in df[df["pickup_name"] == pickup_name].index:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            # name="to ...",
            lat=[df["pickup_latitude"][idx], df["dropoff_latitude"][idx]],
            lon=[df["pickup_longitude"][idx], df["dropoff_longitude"][idx]],
            line=dict(
                # color="#ff0000",
                color=colorFader(c1,c2,np.interp(df["nb"][idx],min_max_nb,[0,1])),
                # colorscale="viridis",
                width=np.interp(df["nb"][idx], min_max_nb, min_max_width),
            ),
        )
    )

fig.update_layout(
    margin={"l": 0, "t": 0, "b": 0, "r": 0},
    mapbox={
        "style": "open-street-map",
        "center": {"lat": 45.5569442, "lon": -73.6336101},
        "zoom": 9.25,
    },
    showlegend=False,
)
fig.show()
